In [1]:
#Required packages
import pandas as pd
import numpy as np 
import pandas as pd 
import re
import string
import nltk 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Merge vertically the three available datasets 
f1 = pd.read_csv('coronavirus-tweet-id-2020-04-23-00.csv')
f2 = pd.read_csv('coronavirus-tweet-id-2020-04-23-01.csv')
f3 = pd.read_csv('coronavirus-tweet-id-2020-04-23-02.csv')
merged = pd.concat([f1, f2, f3])
merged.to_csv('merged.csv', index=None, header=True)

c:\users\maede\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Import the merged dataset
df = pd.read_csv("merged.csv")
print('the size of the original dataset=', df.shape)

#Remove unnecessary columns
data = df.drop(['place','possibly_sensitive','user_created_at','user_default_profile_image', 
                'user_favourites_count','user_listed_count','user_name', 'user_time_zone', 'user_urls'], axis = 'columns')

data.head()

the size of the original dataset= (104402, 34)


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,text,tweet_url,user_screen_name,user_description,user_followers_count,user_friends_count,user_location,user_screen_name.1,user_statuses_count,user_verified
0,NaN,Thu Apr 23 00:00:00 +0000 2020,NaN,NaN,https://www.spokesman.com/stories/2020/apr/21/...,7,1253111343638609922,NaN,NaN,NaN,...,Colleges and universities in Washington and Id...,https://twitter.com/SpokesmanReview/status/125...,SpokesmanReview,"The region’s only daily newspaper, covering ne...",59575,1495,"Spokane, WA",SpokesmanReview,59531,False
1,NaN,Thu Apr 23 00:00:01 +0000 2020,EarthDay FlattenTheCurve COVID19 NYPHeroes Sol...,https://twitter.com/nyphospital/status/1253111...,NaN,53,1253111347614752769,NaN,NaN,NaN,...,"This #EarthDay, we strive to do better for our...",https://twitter.com/nyphospital/status/1253111...,nyphospital,Proud to be the #4 hospital in the nation and ...,48835,4487,"New York, NY",nyphospital,24946,True
2,NaN,Thu Apr 23 00:00:00 +0000 2020,CODVID19 coronovirus scifi amazonbooks,https://twitter.com/MediaTLF/status/1253111342...,https://amazon.com/PHANTOM-APOCALYPSE-Dystopia...,0,1253111342921207808,NaN,NaN,NaN,...,Written before #CODVID19 #coronovirus happened...,https://twitter.com/MediaTLF/status/1253111342...,MediaTLF,A media company that publishes and distributes...,13986,13945,"New York, USA",MediaTLF,12119,False
3,NaN,Thu Apr 23 00:00:00 +0000 2020,LigaMX,NaN,https://chivaspasion.bolavip.com/ligamx/Diario...,2,1253111343693135879,NaN,NaN,NaN,...,🐐🔥 Solo faltaría una aprobación para que la #L...,https://twitter.com/RebanoPasion/status/125311...,RebanoPasion,Sitio no oficial del Guadalajara creado por y ...,38877,56,Guadalajara,RebanoPasion,33281,True
4,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,NaN,0,1253111349040656384,NaN,NaN,NaN,...,RT @StephenNP: Also look first into the nares ...,https://twitter.com/KirklandWalsh/status/12531...,KirklandWalsh,"PhD, FNP-c, GNP-c, specializing in wounds, doc...",720,2171,NaN,KirklandWalsh,95463,False


In [4]:
#Number of rows and columns in data
data.shape

(104402, 25)

In [27]:
#Print the counts of verified and unverified accounts
df['user_verified'].value_counts()

False    102432
True       1970
Name: user_verified, dtype: int64

In [6]:
#Remove verified Twitter accounts
data_unverified = data.drop(data[data['user_verified']== True].index)
data_unverified.shape

(102432, 25)

In [7]:
#Show the number of English and non-English tweets
data_unverified['lang'].value_counts()

en     65283
es     17118
pt      5087
und     3762
ja      3549
in      3263
fr      1207
hi       588
th       474
tl       318
ko       230
tr       216
de       175
ca       169
it       160
ta       127
ru        81
ur        72
zh        71
ar        69
nl        61
ht        42
te        35
ro        29
et        24
mr        22
da        21
sv        20
lt        18
ne        17
pl        15
fa        14
vi         9
cy         9
el         9
cs         8
or         7
lv         6
kn         6
fi         5
eu         5
gu         4
uk         3
ml         3
sl         3
hu         2
no         2
sr         2
si         2
dv         2
iw         2
bn         2
is         1
pa         1
am         1
sd         1
Name: lang, dtype: int64

In [8]:
#Remove non-English tweets
data_unverified_en = data_unverified[data_unverified.lang == 'en']
data_unverified_en.shape

(65283, 25)

In [9]:
data_unverified_en.text[5]

'Crime doesn’t take a break for a pandemic: Coast Guard, FBI still busy // https://t.co/PxhZDO1hlK https://t.co/AsBjb1k4b2'

In [10]:
data_unverified_en.user_location

0                          Spokane, WA
2                        New York, USA
4                                  NaN
5                     Fort Myers, Fla.
7                    Location Location
                      ...             
104395              Fountain Hills, AZ
104397    DC in body/Chicago in spirit
104398                   Milwaukee, WI
104400               Mountain View, CA
104401                             NaN
Name: user_location, Length: 65283, dtype: object

In [11]:
#Split city and country (province) in user_location column  
new = data_unverified_en["user_location"].str.split(",", n = 1, expand = True) 
  
# making separate city column from new data frame 
data_unverified_en["city"]= new[0] 
  
# making separate country or province column from new data frame 
data_unverified_en["country"]= new[1] 

data_unverified_en

<ipython-input-11-0c1542ab5646>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_unverified_en["city"]= new[0]
<ipython-input-11-0c1542ab5646>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_unverified_en["country"]= new[1]


,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_screen_name,user_description,user_followers_count,user_friends_count,user_location,user_screen_name.1,user_statuses_count,user_verified,city,country
0,NaN,Thu Apr 23 00:00:00 +0000 2020,NaN,NaN,https://www.spokesman.com/stories/2020/apr/21/...,7,1253111343638609922,NaN,NaN,NaN,...,SpokesmanReview,"The region’s only daily newspaper, covering ne...",59575,1495,"Spokane, WA",SpokesmanReview,59531,False,Spokane,WA
2,NaN,Thu Apr 23 00:00:00 +0000 2020,CODVID19 coronovirus scifi amazonbooks,https://twitter.com/MediaTLF/status/1253111342...,https://amazon.com/PHANTOM-APOCALYPSE-Dystopia...,0,1253111342921207808,NaN,NaN,NaN,...,MediaTLF,A media company that publishes and distributes...,13986,13945,"New York, USA",MediaTLF,12119,False,New York,USA
4,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,NaN,0,1253111349040656384,NaN,NaN,NaN,...,KirklandWalsh,"PhD, FNP-c, GNP-c, specializing in wounds, doc...",720,2171,NaN,KirklandWalsh,95463,False,NaN,NaN
5,NaN,Thu Apr 23 00:00:00 +0000 2020,NaN,https://twitter.com/winknews/status/1253111342...,https://bit.ly/34XCLO5,1,1253111342283649026,NaN,NaN,NaN,...,winknews,Southwest Florida's News Leader. Have a news t...,89492,688,"Fort Myers, Fla.",winknews,95628,False,Fort Myers,Fla.
7,NaN,Thu Apr 23 00:00:01 +0000 2020,NaN,NaN,NaN,2,1253111348407328768,inminivanhell,1.253084e+18,4.675062e+09,...,kate_barrel,"Books, the outdoors. Community .Gone with the ...",69,98,Location Location,kate_barrel,4240,False,Location Location,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104395,NaN,Thu Apr 23 02:59:59 +0000 2020,NaN,NaN,NaN,0,1253156638359539717,NaN,NaN,NaN,...,shesnottrump,Turn Arizona Blue | Progressive Humanist | #Fu...,1203,582,"Fountain Hills, AZ",shesnottrump,187786,False,Fountain Hills,AZ
104397,NaN,Thu Apr 23 02:59:59 +0000 2020,VeteranOwnedBusiness,NaN,https://twitter.com/Eater_DC/status/1252660118...,0,1253156638598606848,NaN,NaN,NaN,...,xgoddess,Sweets addict and travel lover. Music and socc...,198,777,DC in body/Chicago in spirit,xgoddess,13119,False,DC in body/Chicago in spirit,None
104398,NaN,Thu Apr 23 02:59:59 +0000 2020,NaN,NaN,https://twitter.com/JonahDispatch/status/12529...,0,1253156638632153090,NaN,NaN,NaN,...,tdurkin6,Husband/#GirlDad. @MarquetteU Center for Peace...,634,1220,"Milwaukee, WI",tdurkin6,5510,False,Milwaukee,WI
104400,NaN,Thu Apr 23 02:59:59 +0000 2020,NaN,NaN,https://twitter.com/Reuters/status/12530756819...,0,1253156638384508929,NaN,NaN,NaN,...,eswarpr,Entrepreneur. Entrepreneur’s assistant.,1147,2246,"Mountain View, CA",eswarpr,3050,False,Mountain View,CA


In [12]:
#Convert cities and countries strings to lowercase
data_unverified_en["country"] = data_unverified_en["country"].str.lower()
data_unverified_en["city"] = data_unverified_en["city"].str.lower()

data_unverified_en["country"].value_counts()

<ipython-input-12-d6a97a4397dd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_unverified_en["country"] = data_unverified_en["country"].str.lower()
<ipython-input-12-d6a97a4397dd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_unverified_en["city"] = data_unverified_en["city"].str.lower()


 usa                      4933
 ca                       1502
 india                     880
 tx                        854
 ny                        731
                          ... 
 g a                         1
 where you should be         1
 hamilton, noblesville       1
 yokohama                    1
 ontario, canada             1
Name: country, Length: 2240, dtype: int64

In [13]:
#Select the rows whose country column includes 'CA'  
data_unverified_en_ca = data_unverified_en[data_unverified_en['country']
                   .isin(['ca', ' ca'])]

print(data_unverified_en_ca.shape)
data_unverified_en_ca.city.value_counts()

(1513, 27)


los angeles      360
san francisco    127
san diego         95
sacramento        48
san jose          38
                ... 
fresyes            1
banning            1
alameda            1
vista              1
gardena            1
Name: city, Length: 308, dtype: int64

In [16]:
#Select the rows whose city or counry column is 'Canada'
data_unverified_en_canada = data_unverified_en[(data_unverified_en.city == 'canada') 
                                               | (data_unverified_en.country == 'canada') 
                                               | (data_unverified_en.country == ' canada')]

data_unverified_en_canada.shape       

(568, 27)

In [18]:
#import the 'list_of_cities_Canada' dataset (The reference for cities of Canada)
list_of_cities_Canada = pd.read_csv("list_of_cities_of_canada.csv")
list_of_cities_Canada.head()

,SNo,Province,CityName,Population -2011,Population -2006,Change (%),Land area (km²),Population density (per km²)
0,1.0,Alberta,Airdrie,"42,564","28,927",47.1,33.1,"1,286.00"
1,2.0,Alberta,Brooks,"13,676","12,508",9.3,18.19,751.9
2,3.0,Alberta,Calgary,"10,96,833","9,88,812",10.9,825.29,"1,329.00"
3,4.0,Alberta,Camrose,"17,286","15,630",10.6,42.5,406.7
4,5.0,Alberta,Chestermere,"14,824","9,923",49.4,32.64,454.1


In [19]:
#Convert the city names and provinces of Canada to lowercase in the reference data
list_of_cities_Canada.Province= list_of_cities_Canada.Province.str.lower()
list_of_cities_Canada.CityName= list_of_cities_Canada.CityName.str.lower()

#Make a numpy array from the list of canadian cities/ provinces 
list_Canadian_cities = list_of_cities_Canada.CityName.to_numpy()
list_Canadian_provinces = list_of_cities_Canada.Province.to_numpy()

#Remove duplicates from the provinces list
list_Canadian_provinces = list(dict.fromkeys(list_Canadian_provinces))

['alberta',
 'british columbia',
 'manitoba',
 'new brunswick',
 'newfoundland and labrador',
 'northwest territories',
 'nunavut',
 'ontario',
 'prince edward island',
 'quebec',
 'saskatchewan',
 'yukon',
 nan,
 ' ']

In [24]:
#Select the rows in 'data_unverified_en_ca' dataset whose city is in the list of Canadian cities
data_unverified_en_1 = data_unverified_en_ca[data_unverified_en_ca['city']
                   .isin(list_Canadian_cities)]

#Select the rows in 'data_unverified_en_ca' dataset whose city column value is in the list of Canadian provinces
data_unverified_en_2 = data_unverified_en_ca[data_unverified_en_ca['city']
                   .isin(list_Canadian_provinces)]

In [29]:
#Append the selected rows in data_unverified_en_1 and data_unverified_en_2 datasets to our final dataset (data_unverified_en_canada)
data_unverified_en_canada = data_unverified_en_canada.append(data_unverified_en_1)
data_unverified_en_canada = data_unverified_en_canada.append(data_unverified_en_2)

#our final dataset
data_unverified_en_canada

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,...,user_screen_name,user_description,user_followers_count,user_friends_count,user_location,user_screen_name.1,user_statuses_count,user_verified,city,country
320,NaN,Thu Apr 23 00:00:06 +0000 2020,NaN,NaN,NaN,0,1253111369072869381,NaN,NaN,NaN,...,runcoachphil,"Running Room Regional Manager, Parliament/ Arm...",1023,87,"Ottawa, Canada",runcoachphil,38301,False,ottawa,canada
343,NaN,Thu Apr 23 00:00:06 +0000 2020,NaN,NaN,https://youtu.be/ryx27uJj7xE,1,1253111369387270144,NaN,NaN,NaN,...,LorettaPatriot,🇨🇦 #MCGA 🇺🇸 #MAGA #TRUMP2020 #QAnon #Love #Go...,8609,9469,Canada,LorettaPatriot,82814,False,canada,None
460,NaN,Thu Apr 23 00:00:10 +0000 2020,Covid19,NaN,NaN,0,1253111386198233089,NaN,NaN,NaN,...,brmf_rm,VP RTO D15| Retired OSSTF District President &...,4595,4385,"Ontario, Canada",brmf_rm,39809,False,ontario,canada
498,NaN,Thu Apr 23 00:00:10 +0000 2020,NaN,NaN,NaN,0,1253111386554732546,NaN,NaN,NaN,...,angelasaysmile,news junkie. advocate. mother. living in faith.,242,331,"Ontario, Canada",angelasaysmile,11736,False,ontario,canada
611,NaN,Thu Apr 23 00:00:14 +0000 2020,NaN,NaN,NaN,0,1253111402660651009,NaN,NaN,NaN,...,MtnSounder,"Security expert, long time Poli Sci geek, proj...",246,577,"British Columbia, Canada",MtnSounder,19171,False,british columbia,canada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91254,NaN,Thu Apr 23 02:37:12 +0000 2020,NaN,NaN,https://www.vox.com/first-person/2020/4/22/212...,0,1253150905135845377,NaN,NaN,NaN,...,YoSarahDJ,"Teacher, learner.",170,1257,"Danville, CA",YoSarahDJ,2534,False,danville,ca
10104,NaN,Thu Apr 23 00:21:07 +0000 2020,NaN,NaN,NaN,0,1253116655133233152,NaN,NaN,NaN,...,LDagFrisk,NaN,35,71,"Ontario, CA",LDagFrisk,2271,False,ontario,ca
14992,NaN,Thu Apr 23 00:23:03 +0000 2020,NaN,NaN,NaN,0,1253117144159653888,NaN,NaN,NaN,...,banreportcards,What?Who?Where?When?How?Why? I follow&unfollow...,802,1597,"Ontario, CA",banreportcards,31431,False,ontario,ca
24877,NaN,Thu Apr 23 00:43:48 +0000 2020,NaN,NaN,NaN,0,1253122366852349953,NaN,NaN,NaN,...,MangoNagrik,Not an Engineer! Period. \nRe-tweets are not e...,187,118,"Ontario, CA",MangoNagrik,20114,False,ontario,ca


In [26]:
#Export the final dataset (data_unverified_en_canada) with unverified English tweets from Canada
data_unverified_en_canada.to_csv('data_unverified_en_canada.csv', index=None, header=True)